In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
from torch.utils.data import random_split
from tqdm import tqdm
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)
import random
random.seed(42)


In [21]:

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

transform_train = transforms.Compose([

    transforms.Resize((34, 34)), 
    transforms.RandomCrop(32),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(0.4),
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
    ])


train_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform_train
)

test_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transform_test
)


    

Files already downloaded and verified
Files already downloaded and verified


In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print(batch)

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()


    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

Using cuda device


In [25]:
from torchvision import models
class Resnet18_1(nn.Module):
    def __init__(self):
        super().__init__()
        resnet18_pre = models.resnet18(pretrained=True)
        self.conv_layers = nn.ModuleList()
        for child in resnet18_pre.named_children():
            if child[0] == "fc":
                break
            self.conv_layers.append(child[1])

        self.fc1 = nn.Linear(resnet18_pre.fc.in_features, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):

        for layer in self.conv_layers:
            x = layer(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:

batch_size = 1024
epochs = 50
learning_rate = 1e-3


data = np.array([[0]*4]*epochs)
net_SGD_augm_history = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])



train_dataloader = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True
    )
test_dataloader = DataLoader(
    test_ds, 
    batch_size=batch_size,
    shuffle=False
    )

  #Freeze all layers
net = Resnet18_1().to(device)
for param in net.conv_layers.parameters():    
      param.requires_grad = False


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for t in range(epochs):
    
    print(f"Epoch {t+1}\n-------------------------------")

    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)

    net_SGD_augm_history.loc[t,"train_acc"] = train_acc        
    net_SGD_augm_history.loc[t,"test_acc"] = test_acc
    net_SGD_augm_history.loc[t,"train_loss"] = train_loss
    net_SGD_augm_history.loc[t,"test_loss"] = test_loss       


# resnet1_adam_augm_history.to_csv("resnet1_adam_augm_history.csv")
# !cp resnet1_adam_augm_history.csv "drive/My Drive/DL/augm_add_img_resnet18_1_bs1024_adam/"
 
    

In [ ]:
torch.save(net.state_dict(), '../models/resnet18_1.pth')